In [2]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

train_tfms = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.RandomResizedCrop(300, scale=(0.8, 1.0)),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

valid_tfms = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = ImageFolder("images")




from torch.utils.data import random_split

total_size = len(dataset)
val_size = int(0.2 * total_size)
train_size = total_size - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_ds.dataset.transform = train_tfms
val_ds.dataset.transform = valid_tfms

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

import torch
scaler = torch.amp.GradScaler('cuda')

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    correct = 0
    total = 0
    running_loss = 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        
        with torch.amp.autocast('cuda'):
            logits = model(X)
            loss = criterion(logits, y)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        running_loss += loss.item()
        preds = logits.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return running_loss / len(loader), correct / total


def validate(model, loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)

            logits = model(X)
            loss = criterion(logits, y)

            running_loss += loss.item()
            preds = logits.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return running_loss / len(loader), correct / total

from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(dataset.classes)
weights = EfficientNet_B3_Weights.IMAGENET1K_V1
model = efficientnet_b3(weights=weights)


model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)




In [3]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
best_loss = float('inf')

#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

for epoch in range(15):
    train_loss, train_acc = train_one_epoch(model,train_loader, optimizer,criterion, device)
    val_loss, val_acc = validate(model, valid_loader, criterion, device)
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
    print(f"Epoch:{epoch+1} "
          f"train_loss = {train_loss:.4f}, train_acc = {train_acc:.4f}"
          f"valid_loss = {val_loss:.4f}, val_acc = {val_acc:.4f}")

Epoch:1 train_loss = 3.5507, train_acc = 0.1002valid_loss = 3.4306, val_acc = 0.2963
Epoch:2 train_loss = 3.2810, train_acc = 0.3674valid_loss = 3.0667, val_acc = 0.6288
Epoch:3 train_loss = 2.8157, train_acc = 0.6109valid_loss = 2.4583, val_acc = 0.7786
Epoch:4 train_loss = 2.1869, train_acc = 0.7557valid_loss = 1.7830, val_acc = 0.8428
Epoch:5 train_loss = 1.6277, train_acc = 0.8167valid_loss = 1.2531, val_acc = 0.8736
Epoch:6 train_loss = 1.2173, train_acc = 0.8512valid_loss = 0.9065, val_acc = 0.8930
Epoch:7 train_loss = 0.9629, train_acc = 0.8751valid_loss = 0.6863, val_acc = 0.9017
Epoch:8 train_loss = 0.7792, train_acc = 0.8910valid_loss = 0.5642, val_acc = 0.9110
Epoch:9 train_loss = 0.6653, train_acc = 0.9007valid_loss = 0.4775, val_acc = 0.9130
Epoch:10 train_loss = 0.5548, train_acc = 0.9129valid_loss = 0.4105, val_acc = 0.9151
Epoch:11 train_loss = 0.4887, train_acc = 0.9222valid_loss = 0.3638, val_acc = 0.9184
Epoch:12 train_loss = 0.4365, train_acc = 0.9276valid_loss = 0.